In [1]:
import torch
from torch.nn.utils import clip_grad_value_
# from transformers.models.gemma3.modeling_gemma3 import Gemma3TextScaledWordEmbedding
from functions import *
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-pt-unsloth-bnb-4bit",
    max_seq_length = 8192, # Choose any for long context!
    load_in_4bit = True,
    resize_model_vocab=24,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-03 14:37:01 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.4.3: Fast Gemma3 patching. Transformers: 4.51.3. vLLM: 0.8.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.642 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
model.config.image_token_index = 16 # new image token
model.language_model.lm_head.weight.requires_grad_(True);
# # model.language_model.lm_head.load_state_dict(torch.load("/home/zhenlan/Desktop/Projects/ARC2/Model/gemma24.pth"))
model.language_model.lm_head.load_state_dict(torch.load("../../Model/lm_heads_weights_VLM2.pth"))
model = PeftModel.from_pretrained(model, "../../Model/merged_model_VLM2", is_trainable=False)

In [3]:
model.eval()
model.language_model.model.gradient_checkpointing = False

In [4]:
import json
output_path = '/home/zhenlan/Desktop/Projects/ARC2/Data/ARC-AGI-2-main/combined_data.json'
with open(output_path, 'r') as f:
    data = json.load(f)

In [5]:
inputs_f, y_f = next(iter(data_gen_VLM(data, False, tokenizer, 3, False)))
inputs_t, y_t = next(iter(data_gen_VLM(data, False, tokenizer, 3, True)))
start_idx = len(inputs_t['input_ids'][0]) - 1
end_idx = len(inputs_f['input_ids'][0]) - 2

In [6]:
start_idx, end_idx

(3887, 4756)

In [7]:
# logit1 and logit2 give different results, suggesting something is sequence length dependent.
# tested mannual casual mask, still see the difference. So not caused by mask.
# model.eval();
# with torch.no_grad():
#     with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
#         outputs_full = model(**inputs_f)
#         logits1 = outputs_full.logits[0, :start_idx+1]

# # shorten inputs
# inputs_f['input_ids'] = inputs_f['input_ids'][:, :start_idx+1]
# # inputs_f['attention_mask'] = inputs_f['attention_mask'][:, :, :start_idx+1, :start_idx+1]
# inputs_f['attention_mask'] = inputs_f['attention_mask'][:, :start_idx+1]
# inputs_f['token_type_ids'] = inputs_f['token_type_ids'][:, :start_idx+1]

# with torch.no_grad():
#     with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
#         outputs_full = model(**inputs_f)
#         logits2 = outputs_full.logits
# torch.sum(logits1.argmax(-1) == logits2.argmax(-1))/logits1.shape[0]
# torch.mean(torch.abs(logits1 - logits2)) # original mask, no peft
# torch.mean(torch.abs(logits1 - logits2)) # casual mask
# torch.mean(torch.abs(logits1 - logits2)) # original mask

In [8]:
model.eval();
with torch.no_grad():
    with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
        outputs_full = model(**inputs_f)
        logits1 = outputs_full.logits[0, start_idx: end_idx]

In [9]:
inputs_t['input_ids'][0].shape, inputs_t['attention_mask'].shape

(torch.Size([3888]), torch.Size([1, 1, 3888, 3888]))

In [10]:
# from transformers import HybridCache
# past_key_values = HybridCache(model.config.text_config, 1, 5600)
# logits2 = []

# with torch.no_grad():
#     # Initial pass with the input prompt
#     with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
#         outputs_kv_init = model(**inputs_t, use_cache=True, past_key_values=past_key_values)
#         # outputs_kv_init = model(**inputs_t)
#         logits2.append(outputs_kv_init.logits[0, -1, :])
#         # Store the KV cache from initial pass
#         past_key_values = outputs_kv_init.past_key_values

#     # Iterate through the continuation tokens
#     for i in range(end_idx - start_idx - 1):
#         # Prepare input for the next iteration: only the current continuation token
#         current_token_id_tensor = inputs_f['input_ids'][:, start_idx + i + 1].unsqueeze(1)

#         # Model call with the single next token and past_key_values
#         with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
#             outputs_kv_step = model(input_ids=current_token_id_tensor, 
#                                    past_key_values=past_key_values, 
#                                    use_cache=True,
#                                 #    attention_mask=inputs_f['attention_mask'][:, :start_idx + i + 2], # .unsqueeze(0).unsqueeze(0)
#                                 #    token_type_ids=inputs_f['token_type_ids'][:, start_idx + i + 1:start_idx + i + 2],
#                                    ) # 
#             # Store the logits from this step - CORRECTION HERE
#             logits2.append(outputs_kv_step.logits[0, -1, :])
#             # Update KV cache with new values
#             past_key_values = outputs_kv_step.past_key_values

In [11]:
import torch
from transformers import HybridCache
past_key_values = HybridCache(model.config.text_config, 1, 5600)
# Assume 'model', 'inputs_t', 'inputs_f', 'start_idx', 'end_idx', 'past_key_values'
# are already defined and loaded appropriately.
# 'model' is a Hugging Face transformer model supporting 'cache_position'.
# 'inputs_t' contains the initial prompt tensors (e.g., {'input_ids': tensor, 'attention_mask': tensor})
# 'inputs_f' contains the full sequence including continuation tokens.
# 'past_key_values' might be None initially or contain a pre-filled cache.

logits2 = []
current_seq_len = 0 # Keep track of the sequence length processed so far

with torch.no_grad():
    # --- Initial pass with the input prompt ---
    with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
        outputs_kv_init = model(
            **inputs_t,
            use_cache=True,
            past_key_values=past_key_values,
        )
        # Store the logit for the *last* token of the prompt
        logits2.append(outputs_kv_init.logits[0, -1, :])
        # Store the KV cache from initial pass
        past_key_values = outputs_kv_init.past_key_values
        # Update the sequence length tracker
        current_seq_len = len(inputs_t['input_ids'][0]) # Assuming batch size of 1

    # --- Iterate through the continuation tokens ---
    num_continuation_tokens = end_idx - start_idx - 1
    for i in range(num_continuation_tokens):
        # Prepare input for the next iteration: only the current continuation token
        # Note: Original code uses inputs_f. Ensure this aligns with your logic.
        # Usually, in generation, the input is the *predicted* token from the previous step.
        # Assuming inputs_f contains ground truth continuation here for analysis.
        current_token_id_tensor = inputs_f['input_ids'][:, start_idx + i + 1].unsqueeze(1) # Shape: (batch_size, 1)

        # The position for this single token is the current total sequence length
        # Needs to have shape (batch_size, 1)
        current_cache_position = torch.tensor([current_seq_len], device=current_token_id_tensor.device)

        # Model call with the single next token and past_key_values
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            outputs_kv_step = model(
                input_ids=current_token_id_tensor,
                past_key_values=past_key_values,
                use_cache=True,
                cache_position=current_cache_position # ADDED
            )
            # Store the logits for the token we just processed
            # Logits shape is (batch_size, 1, vocab_size), so index [0, 0, :] or [0, -1, :]
            logits2.append(outputs_kv_step.logits[0, -1, :]) # Index 0 as seq_len is 1
            # Update KV cache with new values
            past_key_values = outputs_kv_step.past_key_values
            # Increment the sequence length tracker
            current_seq_len += 1

# Now logits2 contains the logits for the last token of the prompt
# and each of the subsequent continuation tokens processed one by one.

RuntimeError: The size of tensor a (5600) must match the size of tensor b (3888) at non-singleton dimension 3

In [ ]:
logits2 = torch.stack(logits2, dim=0)

In [ ]:
torch.mean(torch.abs(logits1 - logits2),-1)

tensor([0.1006, 0.0381, 0.0693, 0.0620, 0.0530, 0.0500, 0.0598, 0.0864, 0.0889,
        0.0864, 0.0437, 0.0623, 0.0737, 0.0732, 0.0864, 0.0806, 0.0786, 0.0552,
        0.0598, 0.0623, 0.1104, 0.0933, 0.0669, 0.0869, 0.1016, 0.0771, 0.0757,
        0.0583, 0.0273, 0.0334, 0.0547, 0.0376, 0.0371, 0.0442, 0.0422, 0.0354,
        0.0361, 0.0403, 0.0574, 0.0254, 0.0201, 0.0378, 0.0542, 0.0305, 0.0278,
        0.0869, 0.0444, 0.0574, 0.0432, 0.0574, 0.1025, 0.0840, 0.1011, 0.0879,
        0.0275, 0.0630, 0.0854, 0.0718, 0.0835, 0.0430, 0.0391, 0.0269, 0.0250,
        0.0226, 0.0625, 0.1055, 0.0439, 0.1719, 0.0693, 0.0908, 0.0454, 0.0278,
        0.0732, 0.0952, 0.0559, 0.0583, 0.0398, 0.0718, 0.0625, 0.0649, 0.0854,
        0.1143, 0.0515, 0.0625, 0.0281, 0.0530, 0.0508, 0.0825, 0.1108, 0.1455,
        0.1553, 0.0864, 0.0280, 0.0317, 0.0527, 0.0613, 0.0547, 0.0381, 0.0530,
        0.0601, 0.0737, 0.0498, 0.0430, 0.0352, 0.0281, 0.0244, 0.0359, 0.0284,
        0.0435, 0.0391, 0.0703, 0.0801, 

Text only

In [16]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma3ForConditionalGeneration(
      (vision_tower): SiglipVisionModel(
        (vision_model): SiglipVisionTransformer(
          (embeddings): SiglipVisionEmbeddings(
            (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
            (position_embedding): Embedding(4096, 1152)
          )
          (encoder): SiglipEncoder(
            (layers): ModuleList(
              (0-26): 27 x SiglipEncoderLayer(
                (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
                (self_attn): SiglipAttention(
                  (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
                  (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
                  (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
                  (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
                

In [18]:
model = model.base_model.language_model

In [25]:
model.eval();
with torch.no_grad():
    with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
        outputs_full = model(inputs_f['input_ids'])
        logits1 = outputs_full.logits[0, start_idx: end_idx]

In [20]:
logits2 = []

with torch.no_grad():
    # Initial pass with the input prompt
    with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
        outputs_kv_init = model(inputs_t['input_ids'], use_cache=True)
        logits2.append(outputs_kv_init.logits[0, -1, :])
        # Store the KV cache from initial pass
        past_key_values = outputs_kv_init.past_key_values

    # Iterate through the continuation tokens
    for i in range(end_idx - start_idx - 1):
        # Prepare input for the next iteration: only the current continuation token
        current_token_id_tensor = inputs_f['input_ids'][:, start_idx + i + 1].unsqueeze(1)

        # Model call with the single next token and past_key_values
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            outputs_kv_step = model(input_ids=current_token_id_tensor, 
                                   past_key_values=past_key_values, 
                                   use_cache=True)
            # Store the logits from this step - CORRECTION HERE
            logits2.append(outputs_kv_step.logits[0, -1, :])
            # Update KV cache with new values
            past_key_values = outputs_kv_step.past_key_values

In [21]:
logits2 = torch.stack(logits2, dim=0)

In [26]:
logits2.shape, logits1.shape

(torch.Size([869, 24]), torch.Size([869, 24]))

In [27]:
torch.mean(torch.abs(logits1 - logits2),-1)

tensor([0.0625, 2.7812, 3.1406, 3.1094, 3.0781, 3.0625, 3.1250, 3.1719, 3.2812,
        3.0625, 2.7969, 2.6406, 2.5625, 2.5312, 2.2812, 2.1250, 2.0625, 2.1875,
        2.3906, 2.4062, 2.4062, 2.4062, 2.2969, 2.1406, 2.1094, 2.0625, 1.8359,
        1.6562, 1.6641, 1.6641, 1.9219, 3.0625, 3.6094, 4.5312, 4.8438, 4.9375,
        4.7188, 4.3438, 2.4688, 4.1562, 3.9062, 3.6406, 2.7656, 2.5312, 2.3125,
        2.2344, 1.9766, 1.8984, 1.8594, 1.9219, 2.0938, 1.8828, 1.7422, 1.8906,
        1.9766, 1.9531, 1.9688, 1.8828, 1.5859, 1.2109, 1.4844, 1.9062, 2.3906,
        2.2812, 2.2656, 2.2656, 2.2812, 2.1562, 2.1094, 2.4531, 2.0000, 1.8438,
        1.8594, 1.9922, 1.8984, 1.9531, 1.9531, 1.8047, 1.6797, 1.6562, 1.6016,
        1.5547, 1.7578, 1.6641, 1.7500, 1.6094, 1.5156, 1.5625, 1.4844, 1.5078,
        1.7031, 2.1250, 2.2656, 2.5625, 2.2188, 1.7969, 1.8125, 1.8750, 2.2031,
        2.4062, 2.3125, 2.2031, 2.5781, 2.3125, 2.1094, 2.3438, 2.1094, 2.0781,
        1.8438, 1.7109, 1.9453, 1.6953, 

In [1]:
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration

model = Gemma3ForConditionalGeneration.from_pretrained(
    "google/gemma-3-4b-it",
    # torch_dtype=torch.bfloat16,
    # device_map="auto",
    # attn_implementation="sdpa"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
inputs = torch.randint(0, 1000, (1, 256))#.to("cuda")
start_idx = 128
end_idx = 256

In [3]:
model.eval();
with torch.no_grad():
    with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
        outputs_full = model(inputs)
        logits1 = outputs_full.logits[0, start_idx: end_idx + 1]

In [20]:
from transformers import HybridCache

In [22]:
past_key_values = HybridCache(model.config.text_config, 1, 2048)

In [ ]:
logits2 = []
with torch.no_grad():
    # Initial pass with the input prompt
    with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
        outputs_kv_init = model(inputs[:,:start_idx + 1], use_cache=True, past_key_values=past_key_values)
        logits2.append(outputs_kv_init.logits[0, -1, :])
        # Store the KV cache from initial pass
        past_key_values = outputs_kv_init.past_key_values

    # Iterate through the continuation tokens
    for i in range(end_idx - start_idx - 1):
        # Prepare input for the next iteration: only the current continuation token
        current_token_id_tensor = inputs[:, start_idx + i + 1].unsqueeze(1)

        # Model call with the single next token and past_key_values
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            outputs_kv_step = model(input_ids=current_token_id_tensor, 
                                   past_key_values=past_key_values, 
                                   use_cache=True)
            # Store the logits from this step - CORRECTION HERE
            logits2.append(outputs_kv_step.logits[0, -1, :])
            # Update KV cache with new values
            past_key_values = outputs_kv_step.past_key_values
logits2 = torch.stack(logits2, dim=0)
logits2.shape, logits1.shape

IndexError: index 256 is out of bounds for dimension 1 with size 256

In [26]:
# logits2 = torch.stack(logits2, dim=0)
logits2.shape, logits1.shape

(torch.Size([128, 262208]), torch.Size([128, 262208]))

In [27]:
torch.mean(torch.abs(logits1 - logits2),-1)

tensor([8.5761e-06, 7.2332e-06, 7.8308e-06, 8.9197e-06, 7.5235e-06, 7.1803e-06,
        9.2514e-06, 1.0456e-05, 9.3818e-06, 1.2672e-05, 9.0390e-06, 8.1634e-06,
        7.5472e-06, 1.1036e-05, 1.1434e-05, 1.0017e-05, 9.7087e-06, 1.2099e-05,
        1.3566e-05, 1.0992e-05, 8.9328e-06, 8.0241e-06, 7.5371e-06, 8.4412e-06,
        1.1368e-05, 7.4331e-06, 8.1296e-06, 8.6147e-06, 8.4934e-06, 1.0547e-05,
        9.2572e-06, 7.8645e-06, 9.2196e-06, 1.1559e-05, 9.1299e-06, 1.1141e-05,
        9.9925e-06, 1.1491e-05, 8.4543e-06, 9.1530e-06, 8.3192e-06, 8.1446e-06,
        9.0655e-06, 1.0497e-05, 8.2049e-06, 1.0780e-05, 7.5738e-06, 8.0629e-06,
        6.8507e-06, 1.0084e-05, 9.1416e-06, 8.0334e-06, 8.4851e-06, 1.0729e-05,
        1.3607e-05, 7.8625e-06, 7.1217e-06, 7.0614e-06, 7.1365e-06, 9.5032e-06,
        1.0092e-05, 7.0158e-06, 8.9976e-06, 9.3340e-06, 1.2156e-05, 8.5299e-06,
        9.6471e-06, 1.3323e-05, 1.4118e-05, 1.1182e-05, 7.2470e-06, 9.4420e-06,
        1.4857e-05, 1.0573e-05, 6.9856e-